In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RIStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
1: ((λ#6. (λ#7. #7)) ((λ#8. (#8 #8)) (λ#9. (#9 #9))))
2: ((λ#10. (λ#11. #11)) ((λ#12. (#12 #12)) (λ#13. (#13 #13))))
3: ((λ#14. (λ#15. #15)) ((λ#16. (#16 #16)) (λ#17. (#17 #17))))
4: ((λ#18. (λ#19. #19)) ((λ#20. (#20 #20)) (λ#21. (#21 #21))))
5: ((λ#22. (λ#23. #23)) ((λ#24. (#24 #24)) (λ#25. (#25 #25))))
6: ((λ#26. (λ#27. #27)) ((λ#28. (#28 #28)) (λ#29. (#29 #29))))
7: ((λ#30. (λ#31. #31)) ((λ#32. (#32 #32)) (λ#33. (#33 #33))))
8: ((λ#34. (λ#35. #35)) ((λ#36. (#36 #36)) (λ#37. (#37 #37))))
9: ((λ#38. (λ#39. #39)) ((λ#40. (#40 #40)) (λ#41. (#41 #41))))
10: ((λ#42. (λ#43. #43)) ((λ#44. (#44 #44)) (λ#45. (#45 #45))))
11: ((λ#46. (λ#47. #47)) ((λ#48. (#48 #48)) (λ#49. (#49 #49))))
12: ((λ#50. (λ#51. #51)) ((λ#52. (#52 #52)) (λ#53. (#53 #53))))
13: ((λ#54. (λ#55. #55)) ((λ#56. (#56 #56)) (λ#57. (#57 #57))))
14: ((λ#58. (λ#59. #59)) ((λ#60. (#60 #60)) (λ#61. (#61 #61))))
15: ((λ#62. (λ#63. #63)) ((λ#64. (#64 #64)) (λ#65. (#65 #65))))
16: 

# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#259. (λ#260. (λ#261. ((#259 #260) #261)))) (λ#262. (λ#263. #262))) (λ#264. (λ#265. (#264 (#264 (#264 #265)))))) (λ#266. (λ#267. (#266 #267))))
1: (((λ#268. (λ#269. (((λ#270. (λ#271. #270)) #268) #269))) (λ#272. (λ#273. (#272 (#272 (#272 #273)))))) (λ#274. (λ#275. (#274 #275))))
2: (((λ#276. (λ#277. ((λ#278. #276) #277))) (λ#279. (λ#280. (#279 (#279 (#279 #280)))))) (λ#281. (λ#282. (#281 #282))))
3: (((λ#283. (λ#284. #283)) (λ#285. (λ#286. (#285 (#285 (#285 #286)))))) (λ#287. (λ#288. (#287 #288))))
4: ((λ#289. (λ#290. (λ#291. (#290 (#290 (#290 #291)))))) (λ#292. (λ#293. (#292 #293))))
5: (λ#294. (λ#295. (#294 (#294 (#294 #295)))))


reduction steps: 5
norm term:       (λ#294. (λ#295. (#294 (#294 (#294 #295)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#305. (λ#306. (λ#307. ((#305 #306) #307)))) (λ#308. (λ#309. #309))) (λ#310. (λ#311. #311))) (λ#312. (λ#313. (#312 (#312 (#312 (#312 #313)))))))
1: (((λ#314. (λ#315. (((λ#316. (λ#317. #317)) #314) #315))) (λ#318. (λ#319. #319))) (λ#320. (λ#321. (#320 (#320 (#320 (#320 #321)))))))
2: (((λ#322. (λ#323. ((λ#324. #324) #323))) (λ#325. (λ#326. #326))) (λ#327. (λ#328. (#327 (#327 (#327 (#327 #328)))))))
3: (((λ#329. (λ#330. #330)) (λ#331. (λ#332. #332))) (λ#333. (λ#334. (#333 (#333 (#333 (#333 #334)))))))
4: ((λ#335. #335) (λ#336. (λ#337. (#336 (#336 (#336 (#336 #337)))))))
5: (λ#338. (λ#339. (#338 (#338 (#338 (#338 #339))))))


reduction steps: 5
norm term:       (λ#338. (λ#339. (#338 (#338 (#338 (#338 #339))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#348. (λ#349. (λ#350. (((#348 (λ#351. (λ#352. (#352 (#351 #349))))) (λ#353. #350)) (λ#354. #354))))) (λ#355. (λ#356. #356)))
1: (λ#357. (λ#358. ((((λ#359. (λ#360. #360)) (λ#361. (λ#362. (#362 (#361 #357))))) (λ#363. #358)) (λ#364. #364))))
2: (λ#365. (λ#366. (((λ#367. #367) (λ#368. #366)) (λ#369. #369))))
3: (λ#370. (λ#371. ((λ#372. #371) (λ#373. #373))))
4: (λ#374. (λ#375. #375))


reduction steps: 4
norm term:       (λ#374. (λ#375. #375))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#384. (λ#385. (λ#386. (((#384 (λ#387. (λ#388. (#388 (#387 #385))))) (λ#389. #386)) (λ#390. #390))))) (λ#391. (λ#392. (#391 (#391 (#391 (#391 (#391 #392))))))))
1: (λ#393. (λ#394. ((((λ#395. (λ#396. (#395 (#395 (#395 (#395 (#395 #396))))))) (λ#397. (λ#398. (#398 (#397 #393))))) (λ#399. #394)) (λ#400. #400))))
2: (λ#401. (λ#402. (((λ#403. ((λ#404. (λ#405. (#405 (#404 #401)))) ((λ#406. (λ#407. (#407 (#406 #401)))) ((λ#408. (λ#409. (#409 (#408 #401)))) ((λ#410. (λ#411. (#411 (#410 #401)))) ((λ#412. (λ#413. (#413 (#412 #401)))) #403)))))) (λ#414. #402)) (λ#415. #415))))
3: (λ#416. (λ#417. (((λ#418. ((λ#419. (λ#420. (#420 (#419 #416)))) ((λ#421. (λ#422. (#422 (#421 #416)))) ((λ#423. (λ#424. (#424 (#423 #416)))) ((λ#425. (λ#426. (#426 (#425 #416)))) (λ#427. (#427 (#418 #416)))))))) (λ#428. #417)) (λ#429. #429))))
4: (λ#430. (λ#431. (((λ#432. ((λ#433. (λ#434. (#434 (#433 #430)))) ((λ#435. (λ#436. (#436 (#435 #430)))) ((λ#437. (λ#438. (#438 (#437 #430)))) (λ#439. (#439 ((λ#440. (#440 (#43

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#529. (λ#530. ((#530 (λ#531. (λ#532. (λ#533. (#532 ((#531 #532) #533)))))) #529))) (λ#534. (λ#535. #535))) (λ#536. (λ#537. #537)))
1: ((λ#538. ((#538 (λ#539. (λ#540. (λ#541. (#540 ((#539 #540) #541)))))) (λ#542. (λ#543. #543)))) (λ#544. (λ#545. #545)))
2: (((λ#546. (λ#547. #547)) (λ#548. (λ#549. (λ#550. (#549 ((#548 #549) #550)))))) (λ#551. (λ#552. #552)))
3: ((λ#553. #553) (λ#554. (λ#555. #555)))
4: (λ#556. (λ#557. #557))


reduction steps: 4
norm term:       (λ#556. (λ#557. #557))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#567. (λ#568. ((#568 (λ#569. (λ#570. (λ#571. (#570 ((#569 #570) #571)))))) #567))) (λ#572. (λ#573. (#572 (#572 #573))))) (λ#574. (λ#575. (#574 (#574 (#574 #575))))))
1: ((λ#576. ((#576 (λ#577. (λ#578. (λ#579. (#578 ((#577 #578) #579)))))) (λ#580. (λ#581. (#580 (#580 #581)))))) (λ#582. (λ#583. (#582 (#582 (#582 #583))))))
2: (((λ#584. (λ#585. (#584 (#584 (#584 #585))))) (λ#586. (λ#587. (λ#588. (#587 ((#586 #587) #588)))))) (λ#589. (λ#590. (#589 (#589 #590)))))
3: ((λ#591. ((λ#592. (λ#593. (λ#594. (#593 ((#592 #593) #594))))) ((λ#595. (λ#596. (λ#597. (#596 ((#595 #596) #597))))) ((λ#598. (λ#599. (λ#600. (#599 ((#598 #599) #600))))) #591)))) (λ#601. (λ#602. (#601 (#601 #602)))))
4: ((λ#603. ((λ#604. (λ#605. (λ#606. (#605 ((#604 #605) #606))))) ((λ#607. (λ#608. (λ#609. (#608 ((#607 #608) #609))))) (λ#610. (λ#611. (#610 ((#603 #610) #611))))))) (λ#612. (λ#613. (#612 (#612 #613)))))
5: ((λ#614. ((λ#615. (λ#616. (λ#617. (#616 ((#615 #616) #617))))) (λ#618. (λ#619. (#618 (((λ#620. (λ#62

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#680. (λ#681. ((#681 (λ#682. (λ#683. (λ#684. (((#682 (λ#685. (λ#686. (#686 (#685 #683))))) (λ#687. #684)) (λ#688. #688)))))) #680))) (λ#689. (λ#690. (#689 (#689 (#689 (#689 #690))))))) (λ#691. (λ#692. (#691 #692))))
1: ((λ#693. ((#693 (λ#694. (λ#695. (λ#696. (((#694 (λ#697. (λ#698. (#698 (#697 #695))))) (λ#699. #696)) (λ#700. #700)))))) (λ#701. (λ#702. (#701 (#701 (#701 (#701 #702)))))))) (λ#703. (λ#704. (#703 #704))))
2: (((λ#705. (λ#706. (#705 #706))) (λ#707. (λ#708. (λ#709. (((#707 (λ#710. (λ#711. (#711 (#710 #708))))) (λ#712. #709)) (λ#713. #713)))))) (λ#714. (λ#715. (#714 (#714 (#714 (#714 #715)))))))
3: ((λ#716. ((λ#717. (λ#718. (λ#719. (((#717 (λ#720. (λ#721. (#721 (#720 #718))))) (λ#722. #719)) (λ#723. #723))))) #716)) (λ#724. (λ#725. (#724 (#724 (#724 (#724 #725)))))))
4: ((λ#726. (λ#727. (λ#728. (((#726 (λ#729. (λ#730. (#730 (#729 #727))))) (λ#731. #728)) (λ#732. #732))))) (λ#733. (λ#734. (#733 (#733 (#733 (#733 #734)))))))
5: (λ#735. (λ#736. ((((λ#737. (λ#738. (#737 (

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#845. (λ#846. ((#846 (λ#847. (λ#848. (λ#849. (((#847 (λ#850. (λ#851. (#851 (#850 #848))))) (λ#852. #849)) (λ#853. #853)))))) #845))) (λ#854. (λ#855. (#854 (#854 (#854 #855)))))) (λ#856. (λ#857. (#856 (#856 (#856 (#856 (#856 #857))))))))
1: ((λ#858. ((#858 (λ#859. (λ#860. (λ#861. (((#859 (λ#862. (λ#863. (#863 (#862 #860))))) (λ#864. #861)) (λ#865. #865)))))) (λ#866. (λ#867. (#866 (#866 (#866 #867))))))) (λ#868. (λ#869. (#868 (#868 (#868 (#868 (#868 #869))))))))
2: (((λ#870. (λ#871. (#870 (#870 (#870 (#870 (#870 #871))))))) (λ#872. (λ#873. (λ#874. (((#872 (λ#875. (λ#876. (#876 (#875 #873))))) (λ#877. #874)) (λ#878. #878)))))) (λ#879. (λ#880. (#879 (#879 (#879 #880))))))
3: ((λ#881. ((λ#882. (λ#883. (λ#884. (((#882 (λ#885. (λ#886. (#886 (#885 #883))))) (λ#887. #884)) (λ#888. #888))))) ((λ#889. (λ#890. (λ#891. (((#889 (λ#892. (λ#893. (#893 (#892 #890))))) (λ#894. #891)) (λ#895. #895))))) ((λ#896. (λ#897. (λ#898. (((#896 (λ#899. (λ#900. (#900 (#899 #897))))) (λ#901. #898)) (λ#902. #9

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1894. (λ#1895. ((λ#1896. ((#1896 (λ#1897. (λ#1898. (λ#1899. #1899)))) (λ#1900. (λ#1901. #1900)))) (((λ#1902. (λ#1903. ((#1903 (λ#1904. (λ#1905. (λ#1906. (((#1904 (λ#1907. (λ#1908. (#1908 (#1907 #1905))))) (λ#1909. #1906)) (λ#1910. #1910)))))) #1902))) #1894) #1895)))) (λ#1911. (λ#1912. (#1911 (#1911 (#1911 #1912)))))) (λ#1913. (λ#1914. (#1913 (#1913 #1914)))))
1: (((λ#1915. (λ#1916. ((λ#1917. ((#1917 (λ#1918. (λ#1919. (λ#1920. #1920)))) (λ#1921. (λ#1922. #1921)))) ((λ#1923. ((#1923 (λ#1924. (λ#1925. (λ#1926. (((#1924 (λ#1927. (λ#1928. (#1928 (#1927 #1925))))) (λ#1929. #1926)) (λ#1930. #1930)))))) #1915)) #1916)))) (λ#1931. (λ#1932. (#1931 (#1931 (#1931 #1932)))))) (λ#1933. (λ#1934. (#1933 (#1933 #1934)))))
2: (((λ#1935. (λ#1936. ((λ#1937. ((#1937 (λ#1938. (λ#1939. (λ#1940. #1940)))) (λ#1941. (λ#1942. #1941)))) ((#1936 (λ#1943. (λ#1944. (λ#1945. (((#1943 (λ#1946. (λ#1947. (#1947 (#1946 #1944))))) (λ#1948. #1945)) (λ#1949. #1949)))))) #1935)))) (λ#1950. (λ#1951. (#1950 (#1950 (#1

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2407. (λ#2408. ((λ#2409. ((#2409 (λ#2410. (λ#2411. (λ#2412. #2412)))) (λ#2413. (λ#2414. #2413)))) (((λ#2415. (λ#2416. ((#2416 (λ#2417. (λ#2418. (λ#2419. (((#2417 (λ#2420. (λ#2421. (#2421 (#2420 #2418))))) (λ#2422. #2419)) (λ#2423. #2423)))))) #2415))) #2407) #2408)))) (λ#2424. (λ#2425. (#2424 (#2424 #2425))))) (λ#2426. (λ#2427. (#2426 (#2426 (#2426 (#2426 (#2426 #2427))))))))
1: (((λ#2428. (λ#2429. ((λ#2430. ((#2430 (λ#2431. (λ#2432. (λ#2433. #2433)))) (λ#2434. (λ#2435. #2434)))) ((λ#2436. ((#2436 (λ#2437. (λ#2438. (λ#2439. (((#2437 (λ#2440. (λ#2441. (#2441 (#2440 #2438))))) (λ#2442. #2439)) (λ#2443. #2443)))))) #2428)) #2429)))) (λ#2444. (λ#2445. (#2444 (#2444 #2445))))) (λ#2446. (λ#2447. (#2446 (#2446 (#2446 (#2446 (#2446 #2447))))))))
2: (((λ#2448. (λ#2449. ((λ#2450. ((#2450 (λ#2451. (λ#2452. (λ#2453. #2453)))) (λ#2454. (λ#2455. #2454)))) ((#2449 (λ#2456. (λ#2457. (λ#2458. (((#2456 (λ#2459. (λ#2460. (#2460 (#2459 #2457))))) (λ#2461. #2458)) (λ#2462. #2462)))))) #2448)))) (λ#

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#3445. (λ#3446. (((λ#3447. (λ#3448. ((((λ#3449. (λ#3450. (λ#3451. ((#3449 #3450) #3451)))) #3447) #3448) #3447))) (((λ#3452. (λ#3453. ((λ#3454. ((#3454 (λ#3455. (λ#3456. (λ#3457. #3457)))) (λ#3458. (λ#3459. #3458)))) (((λ#3460. (λ#3461. ((#3461 (λ#3462. (λ#3463. (λ#3464. (((#3462 (λ#3465. (λ#3466. (#3466 (#3465 #3463))))) (λ#3467. #3464)) (λ#3468. #3468)))))) #3460))) #3452) #3453)))) #3445) #3446)) (((λ#3469. (λ#3470. ((λ#3471. ((#3471 (λ#3472. (λ#3473. (λ#3474. #3474)))) (λ#3475. (λ#3476. #3475)))) (((λ#3477. (λ#3478. ((#3478 (λ#3479. (λ#3480. (λ#3481. (((#3479 (λ#3482. (λ#3483. (#3483 (#3482 #3480))))) (λ#3484. #3481)) (λ#3485. #3485)))))) #3477))) #3469) #3470)))) #3446) #3445)))) (λ#3486. (λ#3487. (#3486 (#3486 #3487))))) (λ#3488. (λ#3489. (#3488 (#3488 #3489)))))
1: (((λ#3490. (λ#3491. (((λ#3492. (λ#3493. ((((λ#3494. (λ#3495. (λ#3496. ((#3494 #3495) #3496)))) #3492) #3493) #3492))) (((λ#3497. (λ#3498. ((λ#3499. ((#3499 (λ#3500. (λ#3501. (λ#3502. #3502)))) (λ#3503. (λ#3504.

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#6216. (λ#6217. (((λ#6218. (λ#6219. ((((λ#6220. (λ#6221. (λ#6222. ((#6220 #6221) #6222)))) #6218) #6219) #6218))) (((λ#6223. (λ#6224. ((λ#6225. ((#6225 (λ#6226. (λ#6227. (λ#6228. #6228)))) (λ#6229. (λ#6230. #6229)))) (((λ#6231. (λ#6232. ((#6232 (λ#6233. (λ#6234. (λ#6235. (((#6233 (λ#6236. (λ#6237. (#6237 (#6236 #6234))))) (λ#6238. #6235)) (λ#6239. #6239)))))) #6231))) #6223) #6224)))) #6216) #6217)) (((λ#6240. (λ#6241. ((λ#6242. ((#6242 (λ#6243. (λ#6244. (λ#6245. #6245)))) (λ#6246. (λ#6247. #6246)))) (((λ#6248. (λ#6249. ((#6249 (λ#6250. (λ#6251. (λ#6252. (((#6250 (λ#6253. (λ#6254. (#6254 (#6253 #6251))))) (λ#6255. #6252)) (λ#6256. #6256)))))) #6248))) #6240) #6241)))) #6217) #6216)))) (λ#6257. (λ#6258. (#6257 #6258)))) (λ#6259. (λ#6260. (#6259 (#6259 (#6259 (#6259 #6260)))))))
1: (((λ#6261. (λ#6262. (((λ#6263. (λ#6264. ((((λ#6265. (λ#6266. (λ#6267. ((#6265 #6266) #6267)))) #6263) #6264) #6263))) (((λ#6268. (λ#6269. ((λ#6270. ((#6270 (λ#6271. (λ#6272. (λ#6273. #6273)))) (λ#6274. 